<a href="https://colab.research.google.com/github/212-Sobolev/Math_Prac-1-sem-/blob/hometask_7/Hometask_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача: реализовать и запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета. Добейтесь того, чтобы хотя бы несколько тем были явно интерпретируемы, например, как в примерах ниже.

Примеры топ-10 слов из некотрых тегов, которые получаются после применения LDA:

['god', 'jesus', 'believe', 'life', 'bible', 'christian', 'world', 'church', 'word', 'people'] - эта группа явно соотносится с soc.religion.christian
['drive', 'card', 'hard', 'bit', 'disk', 'scsi', 'memory', 'speed', 'mac', 'video'] - эту группу можно соотнести с темами 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware'
['game', 'games', 'hockey', 'league', 'play', 'players', 'season', 'team', 'teams', 'win'] - тема rec.sport.hockey
Советы:

модель будет сходится лучше и быстрее, если уменьшить размер словаря за счет отсеивания общеупотребительных слов и редких слов. Управлять размером словаря можно с помощью параметров min_df (отсеивает слова по минимальной частоте встречаемости) и max_df (отсеивает слова по максимальной частоте встречаемости) в CountVectorizer.
параметры  α ,  β  можно, для начала, положить единицами
после 100 итераций можно ожидать хорошего распределения по темам. Если этого не происходит и в темах мешинина - проверяйте код и оптимизируйте словарь
на примере третьей темы видно, что у нас встречаются разные формы одного и того же слова. С помощью процедур stemming и lemmatization можно привести слова к общей форме и объединить близкие по значению

In [3]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

# Загрузка NLTK данных (если не загружено)
try:
    nltk.data.find("tokenizers/punkt")
    nltk.data.find("corpora/wordnet")
    nltk.data.find("corpora/stopwords")
    nltk.data.find("tokenizers/punkt_tab")
except LookupError:
    nltk.download("punkt")
    nltk.download("wordnet")
    nltk.download("stopwords")
    nltk.download("punkt_tab")


# Предобработка текста
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token.isalpha()
    ]
    tokens = [token for token in tokens if token not in stop_words]
    return " ".join(tokens)


def lda_gibbs_sampling(
    documents, vocab_size, num_topics, num_iterations, alpha, beta
):
    """
    Реализация LDA с Gibbs Sampling.
    """
    doc_topic_counts = np.zeros((len(documents), num_topics), dtype=int)
    topic_word_counts = np.zeros((num_topics, vocab_size), dtype=int)
    topic_counts = np.zeros(num_topics, dtype=int)
    doc_word_indices = []

    for doc_id, doc in enumerate(documents):
        words = doc.split()
        doc_indices = []
        for word in words:
           try:
             word_id = word_to_id[word]
             doc_indices.append(word_id)
           except:
             pass
        doc_word_indices.append(doc_indices)

    # Инициализация
    doc_topic_assignments = []
    for doc_index, doc in enumerate(doc_word_indices):
      topic_assigments = []
      for word_id in doc:
          topic = np.random.randint(num_topics)
          topic_assigments.append(topic)
          doc_topic_counts[doc_index][topic] += 1
          topic_word_counts[topic][word_id] += 1
          topic_counts[topic] += 1
      doc_topic_assignments.append(topic_assigments)

    # Gibbs Sampling
    for it in range(num_iterations):
        for doc_id, doc in enumerate(doc_word_indices):
          for word_index, word_id in enumerate(doc):
            topic = doc_topic_assignments[doc_id][word_index]

            doc_topic_counts[doc_id][topic] -= 1
            topic_word_counts[topic][word_id] -= 1
            topic_counts[topic] -= 1

            prob = np.zeros(num_topics)
            for k in range(num_topics):
              prob[k] = (doc_topic_counts[doc_id][k] + alpha) * (topic_word_counts[k][word_id] + beta) / (topic_counts[k] + vocab_size * beta)

            prob /= prob.sum()
            new_topic = np.random.choice(num_topics, p=prob)

            doc_topic_assignments[doc_id][word_index] = new_topic
            doc_topic_counts[doc_id][new_topic] += 1
            topic_word_counts[new_topic][word_id] += 1
            topic_counts[new_topic] += 1

    return topic_word_counts

if __name__ == "__main__":
    # Загрузка датасета
    newsgroups = fetch_20newsgroups(
        remove=("headers", "footers", "quotes"), random_state=42
    )
    documents = newsgroups.data

    # Предобработка текста
    processed_documents = [preprocess_text(doc) for doc in documents]

    # Создание матрицы "документ-слово"
    vectorizer = CountVectorizer(min_df=5, max_df=0.95, max_features = 5000)
    X = vectorizer.fit_transform(processed_documents)
    vocab = vectorizer.get_feature_names_out()
    word_to_id = {word: i for i, word in enumerate(vocab)}
    vocab_size = len(vocab)

    # Параметры LDA
    num_topics = 20
    num_iterations = 100
    alpha = 1  # Гиперпараметр Dirichlet prior для тем
    beta = 1  # Гиперпараметр Dirichlet prior для слов

    # Применение LDA с Gibbs Sampling
    topic_word_counts = lda_gibbs_sampling(
        processed_documents, vocab_size, num_topics, num_iterations, alpha, beta
    )

    # Вывод результатов
    print("Топ-10 слов по темам:")
    for topic_id in range(num_topics):
        topic_words = np.argsort(topic_word_counts[topic_id])[::-1][:10]
        top_words = [vocab[word_index] for word_index in topic_words]
        print(f"Тема {topic_id+1}: {top_words}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Топ-10 слов по темам:
Тема 1: ['god', 'one', 'wa', 'jesus', 'christian', 'doe', 'ha', 'say', 'believe', 'bible']
Тема 2: ['wa', 'year', 'game', 'good', 'ha', 'last', 'player', 'team', 'would', 'think']
Тема 3: ['car', 'bike', 'get', 'one', 'engine', 'good', 'much', 'go', 'mile', 'front']
Тема 4: ['space', 'wa', 'launch', 'nasa', 'system', 'satellite', 'mission', 'earth', 'data', 'ha']
Тема 5: ['key', 'chip', 'encryption', 'system', 'use', 'clipper', 'government', 'phone', 'one', 'ha']
Тема 6: ['wa', 'one', 'said', 'people', 'say', 'could', 'know', 'would', 'time', 'go']
Тема 7: ['one', 'price', 'sale', 'used', 'power', 'use', 'good', 'offer', 'box', 'wire']
Тема 8: ['drive', 'card', 'disk', 'system', 'problem', 'driver', 'use', 'mac', 'hard', 'memory']
Тема 9: ['game', 'team', 'play', 'hockey', 'period', 'pt', 'new', 'la', 'wa', 'league']
Тема 10: ['university', 'study', 'research', 'drug', 'health', 'use', 'center', 'new', 'number', 'medical']
Тема 11: ['would', 'know', 'anyone', 'tha